<a href="https://colab.research.google.com/github/DurhamARC/openpose-music/blob/issue4_visualize_output/OpenPose_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


##Install OpenPose

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
  !tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  # --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
  !sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  # CUDA
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
  # CPU 
  #!cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON -DGPU_MODE=CPU_ONLY -DUSE_MKL=OFF .. && make -j`nproc`



##Tests

###Run Python API test examples

In [0]:
!pwd
print("Running Python API example.")
print("The imshow command in that example will lead to a 'cannot connect to X server' error in the end, which can be supressed by changing it to imwrite, for example.")
!cd openpose/build/examples/tutorial_api_python && python3 01_body_from_image.py
print("Another example.")
!cd openpose/build/examples/tutorial_api_python && python3 06_face_from_image.py
print("And another.")
!cd openpose/build/examples/tutorial_api_python && python3 09_keypoints_from_heatmaps.py

### Detect poses on a test video

Connect to Google Drive to access files on which to detect poses.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive/Videos'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Chiranjeeb_raga_1a.mp4	video-clip.mp4


Download the above youtube video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [0]:
!rm -rf video.mp4
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i "/content/drive/My Drive/Videos/NIR_VS_Bhoop_CentralClose_StereoMix.mp4" -t 5 video.mp4
# detect poses on the these 5 seconds
!rm -f openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0 --part-candidates --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

Display the video created by OpenPose:

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii'))) 

# show_local_mp4_video('output.mp4', width=960, height=720)

Create visualisation without background using output json:

In [22]:
!cd '/content/drive/My Drive/openpose-music' && python visualize_sample.py
!ls -l '/content/drive/My Drive/openpose-music'
show_local_mp4_video('/content/drive/My Drive/openpose-music/output.mp4', width=768, height=576)

total 3634
drwx------ 3 root root    4096 May 29 16:28 entimement_openpose
drwx------ 3 root root    4096 May 29 16:28 example_files
-rw------- 1 root root    3549 May 29 14:54 formatted_json.json
-rw------- 1 root root   12133 May 29 16:28 OpenPose_Colab.ipynb
-rw------- 1 root root 1834605 May 29 16:32 output.mp4
-rw------- 1 root root    5686 May 29 14:56 project.avi
-rw------- 1 root root  898901 May 29 15:00 project.mp4
-rw------- 1 root root      16 May 29 16:28 README.md
-rw------- 1 root root  782276 May 22 15:17 rsegray.png
-rw------- 1 root root  172167 May 22 15:13 rse.jpg
-rw------- 1 root root    1170 May 29 16:28 visualize_sample.py
